# Fund Strategy - Factor Strategy 0003

## 1. Import Library

In [1]:
import dai
import random
import pandas as pd

## 2. Prepare Strategy Data

In [2]:
sd = '2022-12-29'
ed = '2026-02-13'

In [3]:
sql_factor = """
SELECT
    date,
    instrument,
    c_pct_rank(-1 * navg_rate) * 0.3 + c_pct_rank(-1 * accum_nav) * 0.4 + c_pct_rank(-1 * unit_nav) * 0.3 AS factor
FROM cn_fund_bar1d JOIN cn_fund_nav USING (date, instrument)
WHERE instrument LIKE '%SH%'
"""

In [9]:
sql_trade = f"""
WITH
data_strategy AS (
    {sql_factor}
),
data_filter AS (
    SELECT
        date,
        instrument,
        factor AS score,
        c_rank(factor) AS score_rank,
    FROM data_strategy
    QUALIFY score_rank <= 5
),
data_date AS (
    SELECT
        date,
        instrument,
        score, 
        score_rank, 
        1 / c_sum(1) AS position, 
    FROM data_filter JOIN mldt_cn_stock_calendar_daily USING (date)
    WHERE is_week_end_trade = 1
)
SELECT *
FROM data_date
ORDER BY date, score_rank
"""

In [10]:
df_strategy = dai.query(sql_trade, filters={"date":[sd, ed]}).df()
df_strategy

,date,instrument,score,score_rank,position
0,2022-12-30,511880.SH,0.008249,1.0,0.2
1,2022-12-30,510090.SH,0.060606,2.0,0.2
2,2022-12-30,510650.SH,0.074411,3.0,0.2
3,2022-12-30,510880.SH,0.077946,4.0,0.2
4,2022-12-30,501046.SH,0.081566,5.0,0.2
...,...,...,...,...,...
800,2026-02-13,511880.SH,0.023741,1.0,0.2
801,2026-02-13,501081.SH,0.041562,2.0,0.2
802,2026-02-13,501077.SH,0.061839,3.0,0.2
803,2026-02-13,561980.SH,0.072670,4.0,0.2


## 3. Backtest Strategy

In [11]:
from bigmodule import M

def BigTrader_Initialize(context):
    from bigtrader.finance.commission import PerOrder
    context.set_commission(PerOrder(buy_cost=0.0003, sell_cost=0.0013, min_cost=5))

def BigTrader_Before_Trading(context, data):
    pass

def BigTrader_Handle_Tick(context, tick):
    pass

def BigTrader_Handle_Data(context, data):

    df_now = context.data[context.data["date"] == data.current_dt.strftime("%Y-%m-%d")]

    if len(df_now) == 0:
        return
    
    target_instruments  = list(df_now["instrument"])
    holding_instruments = list(context.get_account_positions().keys())

    for instrument in holding_instruments:
        if instrument not in target_instruments:
            context.order_target_percent(instrument, 0)
        
    for i, x in df_now.iterrows():
        position = 0.0 if pd.isnull(x.position) else float(x.position)
        context.order_target_percent(x.instrument, position)

def BigTrader_Handle_Trade(context, trade):
    pass

def BigTrader_Handle_Order(context, order):
    pass

def BigTrader_After_Trading(context, data):
    pass

BigTrader = M.bigtrader.v34(
    
    data = df_strategy,
    
    start_date = """""",
    end_date   = """""",
    
    initialize           = BigTrader_Initialize,
    before_trading_start = BigTrader_Before_Trading,
    handle_tick          = BigTrader_Handle_Tick,
    handle_data          = BigTrader_Handle_Data,
    handle_trade         = BigTrader_Handle_Trade,
    handle_order         = BigTrader_Handle_Order,
    after_trading        = BigTrader_After_Trading,
    
    capital_base = 1000000  + random.uniform(0, 10),
    frequency="""daily""",
    product_type="""自动""",
    rebalance_period_type="""交易日""",
    rebalance_period_days="""1""",
    rebalance_period_roll_forward=True,
    backtest_engine_mode="""标准模式""",
    before_start_days=0,
    volume_limit=1,
    order_price_field_buy="""open""",
    order_price_field_sell="""open""",
    benchmark="""沪深300指数""",
    
    plot_charts=True,
    debug=False,
    backtest_only=False,
    m_name="""BigTrader"""
) 

[2026-02-19 14:59:43] [info     ] bigtrader.v34 开始运行 ..
[2026-02-19 14:59:43] [info     ] 2022-12-30, 2026-02-13, , , instruments=84
[2026-02-19 14:59:43] [info     ] bigtrader module V2.2.0
[2026-02-19 14:59:43] [info     ] bigtrader engine v0.1.0.post9+g7a244b6 2026-02-10
order_target_percent: price is null for '501001.SH' at '2023-02-03 15:00:00'
order_target_percent: price is null for '501064.SH' at '2024-04-30 15:00:00'
order_target_percent: price is null for '501064.SH' at '2024-07-26 15:00:00'
order_target_percent: price is null for '501075.SH' at '2025-04-25 15:00:00'
order_target_percent: price is null for '501064.SH' at '2025-12-19 15:00:00'
[2026-02-19 14:59:45] [info     ] backtest done, raw_perf_ds:dai.DataSource("_7461370ba3b6487585a3089e2e9a2c5f")


[2026-02-19 14:59:46] [info     ] bigtrader.v34 运行完成 [3.038s].
